In [1]:
import numpy as np
import random
import sys
sys.path.append('../Util')
sys.path.append('../Wasserstein')

In [3]:
import Hypergraph as hg
import Wasserstein_lp as wlp

In [4]:
def random_simple_graph(num_vertices, p):
    """
    Erdős–Rényi model
    """
    
    vertices = {i: [] for i in range(num_vertices)}
    edges = []
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            rnd = random.random()
            if rnd < p:
                vertices[i].append(j)
                vertices[j].append(i)
                edges.append([i, j])
    
    Graph = {'num_vertices': num_vertices, 
             'num_edges': len(edges), 
             'vertices': vertices, # dictionary of adjacency list
             'edges': edges # list of edges (2-tuples in the form of list)
            }
    
    return Graph
    

In [5]:
Graph = random_simple_graph(5, .5)
for key, value in Graph.items():
    print('{}: {}\n'.format(key, value))

num_vertices: 5

num_edges: 8

vertices: {0: [1, 2, 3, 4], 1: [0, 4], 2: [0, 3, 4], 3: [0, 2, 4], 4: [0, 1, 2, 3]}

edges: [[0, 1], [0, 2], [0, 3], [0, 4], [1, 4], [2, 3], [2, 4], [3, 4]]



In [6]:
connected, dist_matrix = hg.pairwise_distances(Graph['num_vertices'], Graph['edges'])
print('Connected? {}'.format(connected))
print('Pairwise Distance = \n{}\n'.format(dist_matrix))

conditional_probs = hg.hypergraph_random_walk(Graph['num_vertices'], Graph['edges'], lazyness=0)
print('Conditional_probs = \n{}\n'.format(conditional_probs))

Connected? True
Pairwise Distance = 
[[0. 1. 1. 1. 1.]
 [1. 0. 2. 2. 1.]
 [1. 2. 0. 1. 1.]
 [1. 2. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]

Conditional_probs = 
[[0.         0.25       0.25       0.25       0.25      ]
 [0.5        0.         0.         0.         0.5       ]
 [0.33333333 0.         0.         0.33333333 0.33333333]
 [0.33333333 0.         0.33333333 0.         0.33333333]
 [0.25       0.25       0.25       0.25       0.        ]]



In [7]:
def zoom_in(edge, Graph, conditional_probs, dist_matrix):
    center_1 = edge[0]
    center_2 = edge[1]
    nbh_1 = Graph['vertices'][center_1]
    nbh_2 = Graph['vertices'][center_2]
    
    joint_nbh = sorted(list(set(nbh_1).union(set(nbh_2))))
    joint_nbh_map = {}
    for i in range(len(joint_nbh)):
        joint_nbh_map[joint_nbh[i]] = i
    
    num_joint_nbhs = len(joint_nbh)
    
    prob_1 = np.zeros(num_joint_nbhs)
    prob_2 = np.zeros(num_joint_nbhs)
    
    for vertex in nbh_1:
        idx = joint_nbh_map[vertex]
        prob_1[idx] = conditional_probs[center_1][vertex]
    for vertex in nbh_2:
        idx = joint_nbh_map[vertex]
        prob_2[idx] = conditional_probs[center_2][vertex]
    
    return prob_1, prob_2, np.array(dist_matrix)[joint_nbh, : ][:, joint_nbh]  

In [17]:
def Ricci(Graph, *, method='lp'):
    if 'edge_weights' not in Graph.keys():
        weights = np.ones(Graph['num_edges'])
    else:
        weights = Graph['edge_weights']
    
    connected, dist_matrix = hg.pairwise_distances(Graph['num_vertices'], Graph['edges'])
    print('Connected? {}'.format(connected))
    print('Pairwise Distance = \n{}\n'.format(dist_matrix))
    
    if not connected:
        print('The graph is not connected!')
    else:
        conditional_probs = hg.hypergraph_random_walk(Graph['num_vertices'], Graph['edges'], lazyness=0)
        print('Conditional_probs = \n{}\n'.format(conditional_probs))
        
    for edge, weight in zip(Graph['edges'], weights):
        prob_1, prob_2, cost = zoom_in(edge, Graph, conditional_probs, dist_matrix)
        
        if method == 'cvx':
            result = wlp.Wasserstein_dist_cvx(cost, prob_1, prob_2, 1)
        else:
            result = wlp.Wasserstein_dist_lp(cost, prob_1, prob_2).fun
        
        Ricci_cur = 1. - result / weight
        print('{}:\t{}'.format(edge, Ricci_cur))
        

In [13]:
prob_1, prob_2, cost = zoom_in(Graph['edges'][0], Graph, conditional_probs, dist_matrix)
print('prob_1 = {}\nprob_2 = {}\ncost=\n{}'.format(prob_1, prob_2, cost))

prob_1 = [0.   0.25 0.25 0.25 0.25]
prob_2 = [0.5 0.  0.  0.  0.5]
cost=
[[0. 1. 1. 1. 1.]
 [1. 0. 2. 2. 1.]
 [1. 2. 0. 1. 1.]
 [1. 2. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]


In [14]:
result = wlp.Wasserstein_dist_cvx(cost, prob_1, prob_2)
print(result)

0.7499373875573432


In [18]:
Ricci(Graph, method='lp')

Connected? True
Pairwise Distance = 
[[0. 1. 1. 1. 1.]
 [1. 0. 2. 2. 1.]
 [1. 2. 0. 1. 1.]
 [1. 2. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]

Conditional_probs = 
[[0.         0.25       0.25       0.25       0.25      ]
 [0.5        0.         0.         0.         0.5       ]
 [0.33333333 0.         0.         0.33333333 0.33333333]
 [0.33333333 0.         0.33333333 0.         0.33333333]
 [0.25       0.25       0.25       0.25       0.        ]]

[0, 1]:	0.25
[0, 2]:	0.4999999999999998
[0, 3]:	0.4999999999999999
[0, 4]:	0.75
[1, 4]:	0.25
[2, 3]:	0.6666666666666666
[2, 4]:	0.5000000000000002
[3, 4]:	0.5
